# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,callao,-12.0667,-77.1500,20.16,73,75,3.60,PE,1697144147
1,1,bethel,41.3712,-73.4140,21.05,52,0,1.54,US,1697144147
2,2,iskateley,67.6803,53.1512,1.12,76,100,4.02,RU,1697144147
3,3,storvreta,59.9667,17.7000,6.03,70,16,2.57,SE,1697144147
4,4,adamstown,-25.0660,-130.1015,20.84,70,97,5.15,PN,1697144148


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    color = 'City',
    alpha = 0.5,
    frame_height= 500,
    frame_width= 700
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[(city_data_df['Max Temp'] < 35) &(city_data_df['Max Temp'] > 10) &
                            (city_data_df['Humidity'] < 60) &(city_data_df['Humidity'] > 30) &
                            (city_data_df['Wind Speed'] < 7)] 

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,bethel,41.3712,-73.4140,21.05,52,0,1.54,US,1697144147
18,18,hamilton,39.1834,-84.5333,27.58,47,0,4.12,US,1697143645
20,20,invercargill,-46.4000,168.3500,16.68,56,96,6.26,NZ,1697144151
23,23,jamestown,42.0970,-79.2353,16.68,58,75,4.63,US,1697144125
58,58,alofi,-19.0595,-169.9187,26.94,57,40,5.66,NU,1697144159


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,bethel,US,41.3712,-73.4140,52,
18,hamilton,US,39.1834,-84.5333,47,
20,invercargill,NZ,-46.4000,168.3500,56,
23,jamestown,US,42.0970,-79.2353,58,
58,alofi,NU,-19.0595,-169.9187,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit':20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
hamilton - nearest hotel: North Vista Manor
invercargill - nearest hotel: The Grand
jamestown - nearest hotel: DoubleTree Jamestown
alofi - nearest hotel: Matavai Resort
choma - nearest hotel: No hotel found
sabha - nearest hotel: فندق الكاظم-سبها
olavarria - nearest hotel: Santa Rosa Hotel
ghanzi - nearest hotel: Ghanzi farmhouse
port macquarie - nearest hotel: The Observatory
forster - nearest hotel: No hotel found
firuzabad - nearest hotel: هتل جهانگردی
la passe - nearest hotel: L'Estuaire
jaisalmer - nearest hotel: Jeet Mahal
necochea - nearest hotel: Cabañas Río Quequén
papatowai - nearest hotel: No hotel found
puerto deseado - nearest hotel: Los Olmos
hendersonville - nearest hotel: Fairfield Inn & Suites
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
baharly - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
williams lake - nearest hotel: Coast Fraser Inn
awj

,City,Country,Lat,Lng,Humidity,Hotel Name
1,bethel,US,41.3712,-73.4140,52,Hampton Inn Danbury
18,hamilton,US,39.1834,-84.5333,47,North Vista Manor
20,invercargill,NZ,-46.4000,168.3500,56,The Grand
23,jamestown,US,42.0970,-79.2353,58,DoubleTree Jamestown
58,alofi,NU,-19.0595,-169.9187,57,Matavai Resort
...,...,...,...,...,...,...
522,gilgit,PK,35.9221,74.3087,39,PTDC Motel
525,villamontes,BO,-21.2500,-63.5000,36,Hoterma
534,oruro,BO,-17.9833,-67.1500,33,Hotel Edén
546,pimenta bueno,BR,-11.6725,-61.1936,36,Cristalina palace hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
      'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    color = 'City',
    alpha = 0.5,
    frame_height= 500,
    frame_width= 700,
    hover_cols = ['Hotel Name','Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)